In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [8]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [9]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [48]:
result = pd.read_csv('./data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [49]:
def prepare_data(data):
    iw = data.groupby('item_id')['sales_value'].sum().reset_index()
    iw = iw[iw['sales_value'] >= 1]
    iw['weight']= np.log(iw['sales_value'])
    iw['weight']= iw['weight'] / iw['weight'].sum()
    iw.drop('sales_value', axis=1, inplace=True)
    return iw

In [50]:
def weighted_random_recommendation(iw, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
   
    items = iw['item_id'].values
    weights = iw['weight'].values
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [51]:
%%time

item_weights = prepare_data(data_train)

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))

result.head(3)

Wall time: 1.71 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[467401, 5563930, 1067273, 1039301, 13038695]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[10457447, 6919082, 968154, 1015057, 6443283]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]","[8203545, 1136755, 1115428, 920456, 5569471]"


In [52]:
result.to_csv('./data/predictions_basic_2.csv', index=False)

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [57]:
f = lambda r: r.strip('][').split()
fc = lambda r: r.strip('][').split(sep=',')

result = pd.read_csv('./data/predictions_basic_2.csv', quotechar='"', converters={'actual': f, 
'random_recommendation': fc, 'popular_recommendation': fc, 
'itemitem': fc, 'cosine': fc, 'tfidf': fc, 'own_purchases': fc, 'weighted_random_recommendation': fc})

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[467401, 5563930, 1067273, 1039301, 13038695]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[10457447, 6919082, 968154, 1015057, 6443283]"


In [58]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [66]:
for col in result.iloc[:,2:]:
    print("Precision@5 для ", col, ": ","{0:.5f}".format(result.apply
          (lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean()))

Precision@5 для  random_recommendation :  0.00010
Precision@5 для  popular_recommendation :  0.03937
Precision@5 для  itemitem :  0.00784
Precision@5 для  cosine :  0.00901
Precision@5 для  tfidf :  0.00872
Precision@5 для  own_purchases :  0.00000
Precision@5 для  weighted_random_recommendation :  0.00010


Лучшие результаты получились для popular_recommendation.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code